1. Importing required libraries

In [ ]:
from music21 import converter, instrument, note, chord, stream
import glob
import pickle
import numpy as np
from keras.utils import np_utils


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


2. Preprocessing the midi files

In [ ]:
def preprocessMidi(file_path="/content/drive/MyDrive/colab_notebook/Deep_learning/midi_files/*.mid", disp=True):

    arr = []
    for file in glob.glob(file_path):
        midi = converter.parse(file)

        if disp:
            print(f"parsing {file}")

        elements_to_parse = midi.flat.notes


        for ele in elements_to_parse:
            # Checking Notes and storing in the pitch
            if isinstance(ele, note.Note):
                arr.append(str(ele.pitch))

            # checking chord and storing its each note
            elif isinstance(ele, chord.Chord):
                arr.append("+".join(str(n) for n in ele.normalOrder))

    return arr


In [ ]:
noteslis = preprocessMidi()

parsing /content/drive/MyDrive/colab_notebook/Deep_learning/midi_files/Gold_Silver_Rival_Battle.mid
parsing /content/drive/MyDrive/colab_notebook/Deep_learning/midi_files/FF8_Shuffle_or_boogie_pc.mid
parsing /content/drive/MyDrive/colab_notebook/Deep_learning/midi_files/Fyw_piano.mid
parsing /content/drive/MyDrive/colab_notebook/Deep_learning/midi_files/FF4.mid
parsing /content/drive/MyDrive/colab_notebook/Deep_learning/midi_files/FFX_-_Ending_Theme_(Piano_Version)_-_by_Angel_FF.mid
parsing /content/drive/MyDrive/colab_notebook/Deep_learning/midi_files/Ff4-BattleLust.mid
parsing /content/drive/MyDrive/colab_notebook/Deep_learning/midi_files/Fiend_Battle_(Piano).mid
parsing /content/drive/MyDrive/colab_notebook/Deep_learning/midi_files/Finalfantasy6fanfarecomplete.mid
parsing /content/drive/MyDrive/colab_notebook/Deep_learning/midi_files/FF6epitaph_piano.mid
parsing /content/drive/MyDrive/colab_notebook/Deep_learning/midi_files/Kingdom_Hearts_Dearly_Beloved.mid
parsing /content/drive/My

/usr/local/lib/python3.10/dist-packages/music21/midi/translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Martin Rosok Copyright \xa9 1998'>; getting generic Instrument
  warnings.warn(


parsing /content/drive/MyDrive/colab_notebook/Deep_learning/midi_files/ff7themep.mid
parsing /content/drive/MyDrive/colab_notebook/Deep_learning/midi_files/traitor.mid
parsing /content/drive/MyDrive/colab_notebook/Deep_learning/midi_files/thoughts.mid
parsing /content/drive/MyDrive/colab_notebook/Deep_learning/midi_files/ff1battp.mid
parsing /content/drive/MyDrive/colab_notebook/Deep_learning/midi_files/ahead_on_our_way_piano.mid
parsing /content/drive/MyDrive/colab_notebook/Deep_learning/midi_files/Zelda_Overworld.mid
parsing /content/drive/MyDrive/colab_notebook/Deep_learning/midi_files/ff4-town.mid
parsing /content/drive/MyDrive/colab_notebook/Deep_learning/midi_files/lurk_in_dark.mid
parsing /content/drive/MyDrive/colab_notebook/Deep_learning/midi_files/redwings.mid
parsing /content/drive/MyDrive/colab_notebook/Deep_learning/midi_files/mining.mid
parsing /content/drive/MyDrive/colab_notebook/Deep_learning/midi_files/balamb.mid
parsing /content/drive/MyDrive/colab_notebook/Deep_lear

/usr/local/lib/python3.10/dist-packages/music21/midi/translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=6, channel=None, data=b'Pok\xe9mon: Elite Four (Piano)'>; getting generic Instrument
  warnings.warn(


parsing /content/drive/MyDrive/colab_notebook/Deep_learning/midi_files/pkelite4.mid
parsing /content/drive/MyDrive/colab_notebook/Deep_learning/midi_files/dontbeafraid.mid
parsing /content/drive/MyDrive/colab_notebook/Deep_learning/midi_files/ff4-fight1.mid
parsing /content/drive/MyDrive/colab_notebook/Deep_learning/midi_files/relmstheme-piano.mid
parsing /content/drive/MyDrive/colab_notebook/Deep_learning/midi_files/ff6shap.mid
parsing /content/drive/MyDrive/colab_notebook/Deep_learning/midi_files/Still_Alive-1.mid
parsing /content/drive/MyDrive/colab_notebook/Deep_learning/midi_files/tifap.mid
parsing /content/drive/MyDrive/colab_notebook/Deep_learning/midi_files/path_of_repentance.mid
parsing /content/drive/MyDrive/colab_notebook/Deep_learning/midi_files/ff7-mainmidi.mid
parsing /content/drive/MyDrive/colab_notebook/Deep_learning/midi_files/sobf.mid
parsing /content/drive/MyDrive/colab_notebook/Deep_learning/midi_files/electric_de_chocobo.mid
parsing /content/drive/MyDrive/colab_not

3. Storing data in external file

In [ ]:
def storeext(notes,filename='/content/drive/MyDrive/colab_notebook/Deep_learning/mididata.bin'):
    with open(filename, 'wb') as filepath:
        pickle.dump(notes, filepath)

In [ ]:
storeext(noteslis)

In [ ]:
def loadext(filename):
    with open(filename, 'rb') as f:
        notes= pickle.load(f)
        return notes

In [ ]:
notesdata = loadext('/content/drive/MyDrive/colab_notebook/Deep_learning/mididata.bin')
print(f"length of data : {len(notesdata)}")
print(f"Uniques characters : {len(set(notesdata))}")

length of data : 60764
Uniques characters : 398


4. Develpoing model

In [ ]:
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dropout, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow import data

In [ ]:
vocab = sorted(set(notesdata))
char2idx = {c:i for i,c in enumerate(vocab)}
idx2char = {i:c for c,i in char2idx.items()}

seq_length = 100
n_vocab = len(vocab)

def traindata(notes, seq_length):
    input_data = []
    output_data = []

    for i in range(len(notes) - seq_length):
        seq_in = notes[i : i+seq_length] # contains 100 values
        seq_out = notes[i + seq_length]

        input_data.append([char2idx[c] for c in seq_in])
        output_data.append(char2idx[seq_out])

    return input_data, output_data

In [ ]:
input_data, output_data = traindata(notesdata,seq_length=100 )

in_data_len = len(input_data)
print(f"length of input data sequence : {in_data_len}")

input_data = np.reshape(input_data, (in_data_len, seq_length, 1))
print(f"length of reshaped input data : {input_data.shape}")

normalised_input = input_data/float(n_vocab)

length of input data sequence : 60664
length of reshaped input data : (60664, 100, 1)


In [ ]:
network_output = np_utils.to_categorical(output_data)
print(f"Output data lenght : {network_output.shape}")

Output data lenght : (60664, 398)


In [ ]:
model = Sequential()
model.add( LSTM(units=512,
               input_shape = (normalised_input.shape[1], normalised_input.shape[2]),
               return_sequences = True) )
model.add( Dropout(0.3) )
model.add( LSTM(512, return_sequences=True) )
model.add( Dropout(0.3) )
model.add( LSTM(512, return_sequences=True) )
model.add( Dropout(0.3) )
model.add( LSTM(512) )
model.add( Dense(256) )
model.add( Dropout(0.3) )
model.add( Dense(n_vocab, activation="softmax") )

model.compile(loss="categorical_crossentropy", optimizer="adam")
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 512)          1052672   
                                                                 
 dropout (Dropout)           (None, 100, 512)          0         
                                                                 
 lstm_1 (LSTM)               (None, 100, 512)          2099200   
                                                                 
 dropout_1 (Dropout)         (None, 100, 512)          0         
                                                                 
 lstm_2 (LSTM)               (None, 100, 512)          2099200   
                                                                 
 dropout_2 (Dropout)         (None, 100, 512)          0         
                                                                 
 lstm_3 (LSTM)               (None, 512)               2

In [ ]:
checkpoint = ModelCheckpoint("/content/drive/MyDrive/colab_notebook/Deep_learning/models/music_modelv2.hdf5", monitor='loss', verbose=0, save_best_only=True, mode='min')

model_his = model.fit(normalised_input, network_output, epochs=5, batch_size=64, callbacks=[checkpoint])

Epoch 1/5
948/948 [==============================] - 111s 113ms/step - loss: 4.7613
Epoch 2/5
948/948 [==============================] - 106s 112ms/step - loss: 4.7616
Epoch 3/5
948/948 [==============================] - 107s 113ms/step - loss: 4.7609
Epoch 4/5
948/948 [==============================] - 106s 112ms/step - loss: 4.7612
Epoch 5/5
948/948 [==============================] - 107s 113ms/step - loss: 4.7604


In [ ]:
model = load_model("/content/drive/MyDrive/colab_notebook/Deep_learning/models/music_modelv2.hdf5")

Predictions

In [ ]:
def genelement():

  prediction_output = []
  start = np.random.randint(len(input_data) - 1)
  pattern = input_data[start]

  for note_index in range(200):

    prediction_input = np.reshape(pattern, (1, len(pattern), 1)) # convert into numpy desired shape
    prediction_input = prediction_input/float(n_vocab) # normalise

    prediction =  model.predict(prediction_input, verbose=0)

    idx = np.argmax(prediction)
    result = idx2char[idx]
    prediction_output.append(result)

    pattern = input_data[idx]

  return prediction_output

In [ ]:
pred_out = genelement()
print(pred_out)

['F#4', 'B2', 'G5', '6+8', '4+10', 'B-2', 'F4', 'B2', 'G5', '6+8', '4+10', 'B-2', 'F4', 'B2', 'G5', '6+8', '4+10', 'B-2', 'F4', 'B2', 'G5', '6+8', '4+10', 'B-2', 'F4', 'B2', 'G5', '6+8', '4+10', 'B-2', 'F4', 'B2', 'G5', '6+8', '4+10', 'B-2', 'F4', 'B2', 'G5', '6+8', '4+10', 'B-2', 'F4', 'B2', 'G5', '6+8', '4+10', 'B-2', 'F4', 'B2', 'G5', '6+8', '4+10', 'B-2', 'F4', 'B2', 'G5', '6+8', '4+10', 'B-2', 'F4', 'B2', 'G5', '6+8', '4+10', 'B-2', 'F4', 'B2', 'G5', '6+8', '4+10', 'B-2', 'F4', 'B2', 'G5', '6+8', '4+10', 'B-2', 'F4', 'B2', 'G5', '6+8', '4+10', 'B-2', 'F4', 'B2', 'G5', '6+8', '4+10', 'B-2', 'F4', 'B2', 'G5', '6+8', '4+10', 'B-2', 'F4', 'B2', 'G5', '6+8', '4+10', 'B-2', 'F4', 'B2', 'G5', '6+8', '4+10', 'B-2', 'F4', 'B2', 'G5', '6+8', '4+10', 'B-2', 'F4', 'B2', 'G5', '6+8', '4+10', 'B-2', 'F4', 'B2', 'G5', '6+8', '4+10', 'B-2', 'F4', 'B2', 'G5', '6+8', '4+10', 'B-2', 'F4', 'B2', 'G5', '6+8', '4+10', 'B-2', 'F4', 'B2', 'G5', '6+8', '4+10', 'B-2', 'F4', 'B2', 'G5', '6+8', '4+10', 'B-2'

In [ ]:
def pred_to_notes(prediction_output):

  offset = 0 # Time
  output_notes = []

  for pattern in prediction_output:

      # if the pattern is a chord
      if ('+' in pattern) or pattern.isdigit():
          notes_in_chord = pattern.split('+')
          temp_notes = []
          for current_note in notes_in_chord:
              new_note = note.Note(int(current_note))  # create Note object for each note in the chord
              new_note.storedInstrument = instrument.Piano()
              temp_notes.append(new_note)


          new_chord = chord.Chord(temp_notes) # creates the chord() from the list of notes
          new_chord.offset = offset
          output_notes.append(new_chord)

      else:
              # if the pattern is a note
          new_note = note.Note(pattern)
          new_note.offset = offset
          new_note.storedInstrument = instrument.Piano()
          output_notes.append(new_note)

      offset += 0.5

  return output_notes

In [ ]:
midi_stream = stream.Stream(pred_to_notes(pred_out))
midi_stream.write('midi', fp = "test_output.mid")

'test_output.mid'

In [ ]:
midi_stream.show('midi')